In [6]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision
from copy import deepcopy
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch.utils.data.sampler import RandomSampler

ModuleNotFoundError: No module named 'torchvision'

In [7]:
pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.1/161.1 kB 10.9 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


## 初始化参数

In [11]:
n_fold = 5
pad_left = 27
pad_right = 27
fine_size = 202
batch_size = 18
epoch = 300 
snapshot = 6 
max_lr = 0.012 
min_lr = 0.001 
momentum = 0.9 
weight_decay = 1e-4 

### GPU运行工具，后面会用

In [12]:
device = torch.device('cuda')

### 创建根目录，利用跟目录找到其他目录

In [18]:
src = f'F:/kaggle/competition_data'
save_weight_path = src + '/weight'
train_image_dir = src + '/train/images'
train_mask_dir = src + '/train/masks'
test_image_dir = src + '/test/images'

1. 读取depth数据
2. 按z的大小排序，替换
3. 删除z列，替换
4. 分组


In [22]:
depths = pd.read_csv(src + '/depths.csv')
depths.sort_values('z', inplace=True)
depths.drop('z', axis=1, inplace=True)
depths['fold'] = (list(range(0,5)) * depths.shape[0])[:depths.shape[0]]

1. 读取train的数据
2. 合并数据
3. 创建空数列dist
4. 读取图片，灰度图（101，101）
5. 计算图片不同像素值个数
6. 存入dist

In [25]:
train_df = pd.read_csv(src + '/train.csv')
train_df = train_df.merge(depths)
dist = []

for id in train_df.id.values:
  img = cv2.imread(train_image_dir+f'/{id}.png', cv2.IMREAD_GRAYSCALE)
  dist.append(np.unique(img).shape[0])
train_df['unique_pixels'] = dist

1. 获取id列表
2. 根据fold对数据进行分组（5组）

In [27]:
all_id = train_df['id'].values
fold = []
for i in range(5):
  fold.append(train_df.loc[train_df['fold']==i, 'id'].values)